In [ ]:
import os
import numpy as np
from scipy.stats import poisson
import sqlite3
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
folder_db = '/data/databases/'
folder_raw = '/data/raw_files/'

# Get the directory of the current script
current_directory = os.path.abspath('')


db_path = os.path.join(current_directory + folder_db, "fbref_data_team_individual.db")
conn = sqlite3.connect(db_path)
df_individual_xpts = pd.read_sql_query('SELECT * FROM team_individual_tbl', conn)
conn.close()
df_individual_xpts = df_individual_xpts[df_individual_xpts['year'] == '2023-2024']
df_individual_xpts.tail(15)

In [ ]:
# Same simulate_match function as before
def simulate_match(xG_home, xG_away, num_simulations=50000):
    home_goals = poisson.rvs(mu=xG_home, size=num_simulations)
    away_goals = poisson.rvs(mu=xG_away, size=num_simulations)
    
    home_wins = np.sum(home_goals > away_goals)
    draws = np.sum(home_goals == away_goals)
    
    win_probability_home = home_wins / num_simulations
    draw_probability = draws / num_simulations
    
    xPts_home = (win_probability_home * 3.0) + (draw_probability * 1.0)
    return xPts_home

# Applying the simulate_match function to each row in the dataframe
df_individual_xpts['xPts'] = df_individual_xpts.apply(lambda row: simulate_match(row['xg'], row['xga']), axis=1)

df_individual_xpts.tail(10)


In [ ]:
df_individual_xpts[df_individual_xpts['team'] == 'Tottenham']

In [ ]:
# Group by 'team' and sum 'xPts', then sort by the xPts values
team_xPts = df_individual_xpts.groupby('team')['xPts'].sum().sort_values(ascending=False)

# Convert the Series to a DataFrame
team_xPts_df = team_xPts.reset_index()
team_xPts_df.loc[team_xPts_df['team']=='Everton', 'xPts'] -= 10

# Add a new 'Rank' column which provides a rank from 1 to the length of the DataFrame
team_xPts_df['Rank'] = team_xPts_df['xPts'].rank(ascending=False, method='min').astype(int)

print(team_xPts_df)


In [ ]:
url = 'https://fbref.com/en/comps/9/Premier-League-Stats'
response = requests.get(url).text.replace('<!--', '').replace('-->', '')
df_tables = pd.read_html(response)
df_tables_prem = df_tables[0]
cols = ['Rk', 'Squad', 'Pts']
df_tables_prem = df_tables_prem[cols]
# Display the first table from the fetched tables
print(df_tables_prem)


In [ ]:
merged_df = pd.merge(df_tables_prem, team_xPts_df, left_on='Squad', right_on='team', how='inner')
merged_df

In [ ]:
merged_df.to_csv('data/raw_files/tables_expected_actual.csv')